# Fuzzy Matching with RL - 247 & Rivals

> Leveraging the RL library to determine approximate matching over a range of fields using various string methods methods.  This specifically focuses on 247 & Rivals first.

In [1]:
import json
import pandas
import time
import os
import recordlinkage
import csv
import core_constants as cc
import functions as fx
import sqlite3 as sql

#not currently using jellyfish
import jellyfish as jf

## Load and Merge Source Files Then Create a List of Dicts for each Dataset
> This was originally set up for all of the keys in the sourcefiles.json config.  Since this file is currently only going to serve 247 & Rivals, I've hardcoded the keys to fetch

In [2]:
dfFinal = fx.doFuzzyMatching('NCAA', 'Sports247')
dfFinal

,sourceID,targetID,ID,PlayerName,Position,Year,sum
"(aaronburks_boisestate, aaronburkes_boisestate_2009)",aaronburks_boisestate,aaronburkes_boisestate_2009,0.954545,0.909091,0.25,0.00,0.528409
"(aarondopson_marshall, aarondobson_marshall_2009)",aarondopson_marshall,aarondobson_marshall_2009,0.950000,0.909091,0.00,0.00,0.464773
"(aarongreen_tcu, aaronbrown_tcu_2005)",aarongreen_tcu,aaronbrown_tcu_2005,0.785714,0.700000,0.25,0.00,0.433929
"(aaronhepp_ballstate, aaronclay_ballstate_2013)",aaronhepp_ballstate,aaronclay_ballstate_2013,0.789474,0.555556,0.25,0.25,0.461257
"(aaronwilliams_northwestern, traewilliams_northwestern_2015)",aaronwilliams_northwestern,traewilliams_northwestern_2015,0.846154,0.692308,0.00,0.25,0.447115
...,...,...,...,...,...,...,...
"(zechmcphearson_pennstate, zechariahmcphearson_pennstate_2016)",zechmcphearson_pennstate,zechariahmcphearson_pennstate_2016,0.827586,0.736842,0.00,0.25,0.453607
"(zekechapman_southalabama, dezekiuschapman_southalabama_2020)",zekechapman_southalabama,dezekiuschapman_southalabama_2020,0.821429,0.666667,0.00,0.25,0.434524
"(zenohannibal_louisianatech, zachhannibal_louisianatech_2016)",zenohannibal_louisianatech,zachhannibal_louisianatech_2016,0.884615,0.750000,0.00,0.25,0.471154
"(zionjohnson_bostoncollege, samjohnson_bostoncollege_2019)",zionjohnson_bostoncollege,samjohnson_bostoncollege_2019,0.840000,0.636364,0.00,0.25,0.431591


In [3]:
conn = sql.connect(cc.databaseName) 
          
sql_query = pandas.read_sql_query ('''
                               SELECT
                               *
                               FROM SourcedPlayers
                               WHERE KeyDataSet = 1
                               ''', conn)

df_247 = pandas.DataFrame(sql_query, columns = ['IDYR', 'College', 'Year', 'Position'])
df_247.set_index('IDYR', append=False, inplace=True)
sql_query = pandas.read_sql_query ('''
                               SELECT
                               *
                               FROM UnlinkedNCAA
                               ''', conn)

df_NCAA = pandas.DataFrame(sql_query, columns = ['ID', 'College', 'Year', 'Position'])
df_NCAA.set_index('ID', append=False, inplace=True)

#dfFinal.reset_index(drop=True, inplace=True)
#dfFinal.set_index(['sourceID', 'targetID'], append=False, inplace=True)

dfFinal.head()

,sourceID,targetID,ID,PlayerName,Position,Year,sum
"(aaronburks_boisestate, aaronburkes_boisestate_2009)",aaronburks_boisestate,aaronburkes_boisestate_2009,0.954545,0.909091,0.25,0.00,0.528409
"(aarondopson_marshall, aarondobson_marshall_2009)",aarondopson_marshall,aarondobson_marshall_2009,0.950000,0.909091,0.00,0.00,0.464773
"(aarongreen_tcu, aaronbrown_tcu_2005)",aarongreen_tcu,aaronbrown_tcu_2005,0.785714,0.700000,0.25,0.00,0.433929
"(aaronhepp_ballstate, aaronclay_ballstate_2013)",aaronhepp_ballstate,aaronclay_ballstate_2013,0.789474,0.555556,0.25,0.25,0.461257
"(aaronwilliams_northwestern, traewilliams_northwestern_2015)",aaronwilliams_northwestern,traewilliams_northwestern_2015,0.846154,0.692308,0.00,0.25,0.447115


In [4]:
miFinal = pandas.MultiIndex.from_frame(dfFinal)
miFinal

MultiIndex([(              'aaronburks_boisestate', ...),
            (               'aarondopson_marshall', ...),
            (                     'aarongreen_tcu', ...),
            (                'aaronhepp_ballstate', ...),
            (         'aaronwilliams_northwestern', ...),
            (      'acoryewhite_alabamabirmingham', ...),
            (             'adamharter_kansasstate', ...),
            (                    'adamplantjr_tcu', ...),
            (            'adamtaylor_northwestern', ...),
            ('adarriusperkins_southernmississippi', ...),
            ...
            (         'zacktaylor_westernmichigan', ...),
            (         'zaireanderson_northwestern', ...),
            (                'zameljohnson_temple', ...),
            (                'zaneporter_marshall', ...),
            (              'zanewilliams_kentucky', ...),
            (           'zechmcphearson_pennstate', ...),
            (           'zekechapman_southalabama', ...)

In [5]:
recordlinkage.write_annotation_file(
    "annotation_ncaa.json",
    miFinal[0:1300],
    df_NCAA,
    df_247,
    dataset_a_name="NCAA",
    dataset_b_name="Master"
)

In [ ]:
dfFinal.head()

## Convert Series to Dataframe
> output of the above is a Pandas Series and needs to be pushed to a dataframe to be accessible

In [ ]:
dfFinal = pandas.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.sort_values(by='SUM')

In [ ]:
dict_dfFinal = dfFinal.to_dict('records')

In [ ]:
dict_dfFinal[0]

In [ ]:
for record in dict_dfFinal:
    Values = [record['sourceID'], record['targetID'], 5, 3, record['sum']]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence)
        VALUES (?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()